# [Module 4] Interactions, Items, Users 데이터셋을 S3에 업로드

이 워크샵은 기본 커널을 conda_python3를 사용합니다.

## 0. 환경 설정

#### Library Import 

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 LAB을 위해서 핵심 Data Scientist용 Tool 인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [ ]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt

변수 로딩

In [ ]:
%store -r

## 1. 데이터셋 확인

In [ ]:
interactions_df.head(5)

In [ ]:
items_df.head(5)

In [ ]:
users_df.head(5)

## 2. 데이터 분리

이 데이터에는 UserID, ItemID, Event_Type 및 Timestamp 컬럼이 포함되어 있습니다.<br>

- 시간순으로 정렬한 후에 90%는 학습용 데이터로, 마지막 최근 10% 데이터를 검증용 데이터로 분리합니다.

#### Data Set을 Train, Validation(holdout) 데이터 분리하기 

모든 사용자의 마지막(Timestamp기준으로) 10%의 데이터를 Validation(Holdout) 데이터로 분리합니다.

In [ ]:
pd.options.display.max_rows = 5
def split_holdout(data, pct):
    df = data.copy()
    # Rank per each subgroup, 'USER_ID'
    ranks = df.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
    df = df.join((ranks> pct).to_frame('holdout'))
    
    holdout = df[df['holdout']].drop('holdout', axis=1)
    train = df[~df['holdout']].drop('holdout', axis=1)    
    
    return train, holdout

df_warm_train, df_warm_holdout = split_holdout(interactions_df, pct=0.9)

train 과 holdout의 분리된 데이타의 정보입니다. holdout은 대략 전체 대비 10%의 Row의 갯수 입니다. holdout의 timestamp는 train보다 미래의 숫자인 것을 인지할 수 있습니다. (숫자가 많은 것이 더 미래의 날짜를 의미함)

In [ ]:
df_warm_train.info()
df_warm_train.nunique()

In [ ]:
df_warm_holdout.info()
df_warm_holdout.nunique()

In [ ]:
df_warm_train.sort_values(['USER_ID','TIMESTAMP'])

In [ ]:
df_warm_holdout.sort_values(['USER_ID','TIMESTAMP'])

## 3. 로컬에 train, item, validation(holdout) 및 coldstart 를 CSV 저장

완료되면 파일을 새 CSV로 저장한 다음, S3에 업로드합니다.<br>

In [ ]:
import os
os.makedirs('base-dataset', exist_ok=True)

train, item, validation(holdout) 데이터를 로컬에 csv 파일로 저장 합니다.

In [ ]:
# train, item, validation 로컬 저장
base_warm_train_interaction_filename="base-dataset/training_interaction.csv"
base_items_filename="base-dataset/training_item.csv"
base_users_filename="base-dataset/training_user.csv"
base_validation_interaction_filename="base-dataset/validation_interaction.csv"

df_warm_train.to_csv(base_warm_train_interaction_filename,index=False)
items_df.to_csv(base_items_filename,index=False)
users_df.to_csv(base_users_filename,index=False)
df_warm_holdout.to_csv(base_validation_interaction_filename,index=False)    

In [ ]:
# warm data set 로컬 저장
base_warm_interation_filename="base-dataset/warm_interaction.csv"
interactions_df.to_csv(base_warm_interation_filename,index=False)


## 4. S3에 로컬 CSV 업로드

In [ ]:
import sagemaker
#bucket='<YOUR BUCKET NAME>' # replace with the name of your S3 bucket
bucket = sagemaker.Session().default_bucket() 

In [ ]:
#upload file for training
response_upload = boto3.Session().resource('s3').Bucket(bucket).Object(base_warm_train_interaction_filename).upload_file(base_warm_train_interaction_filename)
boto3.Session().resource('s3').Bucket(bucket).Object(base_users_filename).upload_file(base_users_filename)
boto3.Session().resource('s3').Bucket(bucket).Object(base_items_filename).upload_file(base_items_filename)

s3_base_warm_train_interaction_filename = "s3://{}/{}".format(bucket, base_warm_train_interaction_filename)
s3_base_items_filename = "s3://{}/{}".format(bucket, base_items_filename)
s3_base_users_filename = "s3://{}/{}".format(bucket, base_users_filename)

print("s3_base_warm_train_interaction_filename: \n", s3_base_warm_train_interaction_filename)
print("s3_base_items_filename: \n", s3_base_items_filename)
print("s3_base_users_filename: \n", s3_base_users_filename)

In [ ]:
! aws s3 ls {s3_base_warm_train_interaction_filename} --recursive
! aws s3 ls {s3_base_items_filename} --recursive
! aws s3 ls {s3_base_users_filename} --recursive

## 5. 변수 저장

다음 노트북에서 활용할 변수를 저장 합니다.

In [ ]:
%store bucket

%store s3_base_warm_train_interaction_filename
%store s3_base_users_filename
%store s3_base_items_filename
%store base_warm_train_interaction_filename
%store base_items_filename
%store base_users_filename
%store base_validation_interaction_filename
%store base_warm_interation_filename
